<center><h1>Generalized Linear Models in R</h1></center>

# 1. Generalized Linear Models

  - Family of models _including_ linear models
  - Generalization of linear models to included categorical and/or non-normaly distributed dependent/outcome variable
  - Examples:
    + Binomial logistic regression
    + Multinomial regression
    + Poisson regression

 ## 1.1 Note on Terminology
  - General Linear Model $\ne$ General**ized** Linear Model

  - General linear model refers to models with a continuous outcome variable, and assumption of normality
    + ANOVA (and friends)
    + Linear regression

  - Term General**ized** Linear Model used to refer to a family of models for categorical and/or non-normally distributed outcome variables 
      

# 2 Binomial Logistic Regression

  - Linear regression assumes a continuous outcome variable

  - If the outcome variable is _not_ continuous, we need a different approach. 
        
  - In the case of a binary outcome variable, we model $\text{Pr}(y_i = 1)$


## 2.1 Logistic Regression vs. Linear Regression
    
Differences from linear regression:
  - Assumes outcome is bounded by $0$ and $1$
    + that is $0 \le E(y_i) = \pi_i \le 1$
  - Variance of $y$ is _not_ constant (i.e., not the same for all $y_i$)
  - Similarly, variance of $\varepsilon$ is not constant
  - Computational differences (i.e., closed-form vs numerical methods)


## 2.2 Components of Generalized Linear Models
Recall the form of the linear model:
        
$$ y = \beta_0 + \beta_1 x_1 + ... + \beta_p x_p + \varepsilon $$ 
        

which can also be written in matrix notation as
        
$$\mathbf{y} = \mathbf{X} \boldsymbol{\beta} + \boldsymbol{\varepsilon} $$
        
where $\mathbf{X} \boldsymbol{\beta}$ is the systemic component and $\boldsymbol{\varepsilon}$ is the random component.


### 2.2.1 Components of Generalized Linear Model (cont.)

Form of GLM:
$$g(\mu) = \mathbf{X} \boldsymbol{\beta}$$

Generalized linear models have 3 components:
1. Systemic component
  - Same as linear regression (e.g., $\mathbf{X} \boldsymbol{\beta}$)
2. Response distribution assumption
  - Random component of the model
  - Specifies the probabilistic mechanism by which responses were generated
3. Link function
  - This is $g(\cdot)$ in equation above


### 2.2.2 The Link Function

Link function is a characteristic feature of generalized linear models

A link function:
1. Connects the systemic component to response (i.e., "links" them) 
 - Allows us to map a linear function with range $\left( - \infty, \infty \right)$ to some new range; e.g., $\left(0, 1\right)$
2. Differs according to the species of GLM in question (and even within)
3. Similar to "activation functions" in artificial neural networks


### 2.2.3 Binomial Logistic Regression
Logistic regression with a single predictor:

\begin{equation*}
\begin{split}
\pi(x_1) & = \frac{\exp{(\beta_0 + \beta_1 x_1 )}}{1 + \exp{(\beta_0 + \beta_1 x_1 )}} \\
& \\
& = \frac{1}{1 + \exp{(- \eta )}}
\end{split}
\end{equation*}
where $\eta = \beta_0 + \beta_1 x_1$

### 2.2.4 Binomial Logistic Regression
$$ \pi(x_1) = \frac{\exp{(\beta_0 + \beta_1 x_1 )}}{1 + \exp{(\beta_0 + \beta_1 x_1 )}} $$


Note that the $ \beta_0 + \beta_1 x_1 $ in the above equation is the same as we saw in linear regression. This is called the _linear predictor_ in logistic regression

### 2.2.5 Interpreting Parameter Estimates
Interpretation of logistic regression parameter estimates:


1. Slightly different than linear regression
2. Recall our model is $\text{Pr}\left(y_i = 1\right) = \text{logit}^{-1}\left(\mathbf{X} \boldsymbol{\beta}\right)$
3. Regression parameters estimates are on logit scale (log odds), 
  - It's common to exponentiate $\widehat{\boldsymbol{\beta}}$
  - Value of $\text{exp}\left(\beta_j\right)$ is the odds ratio of 1-unit increase on $x_j$

### 2.2.6 Model Evaluation

1. Recall that $R^2$ in linear regression gives us a nice method of evaluating models (i.e., proportion of variance explained). 
2. However, in logistic regression, there is no direct analogue to $R^2$ (but there are some similar measure)
3. Thus, we tend to rely on the information-based criteria discussed previously (e.g., AIC, BIC)
  - These also have the advantage of penalizing unnecessary model complexity

### 2.2.7 Choice of Link Function
Several link function options for modeling binomial data:

1. Logit link (most common, by far)
2. CDF of normal distribution (probit regression)
3. CDF of $t$-distribution (_robit_ model; robust binomial regression)
  - Degrees of freedom parameter allows for flexibility in accommodating outliers

### 2.2.8 When to use Logistic Regression

  - Used when outcome variable takes one of two values (e.g., $0$ or $1$, "lived" or "died") 
  - Similar structure as linear regression 

    + Estimate effects of predictors on outcome
    + Can have one or many predictors

  - Can answer similar kinds of questions as linear regression, for example:
    + _What is the effect of the predictor, $x$, on the outcome $y$?_

## 2.3 Single-Variable Logistic Regression in R

  - Suppose we are interested in the survival of passengers on the Titanic. 
  - In particular, suppose we want to know whether a passenger's gender impacted whether or not they survived.
  - We can investigagte this using logistic regression.

In [ ]:
library(broom)

titanic_df <- read.csv("data/titanic_subset.csv")

head(titanic_df)

### 2.3.1 Fitting the Model
- Generalized linear models are fitted using `glm()` function

In [ ]:
fm1 <- glm(survived ~ sex, titanic_df, family = binomial(link = "logit"))

In [ ]:
tidy(fm1)           # show table of regression parameter estimates

glance(fm1)         # show fit indices

### 2.3.2 Using Fitted Model to Make Predictions

- We can use fitted model object (e.g., `fm1`) to make predictions for new data
- Use the `predict()` function

In [ ]:
predict(fm1, newdata = data.frame(sex = "male"), type = "response")

## 3. Why do we Need Logistic Regression?

In [ ]:
fm1b <- lm(survived ~ sex, titanic_df)

In [ ]:
tidy(fm1b)

## 3.1 Simulated Data Example

In [ ]:
n <- 1500                           # set sample size
beta0 <- 1.2                        # specify regression parameters
beta1 <- -0.7

x <- rnorm(n)           

invlogit <- function(eta) {         # Define a function to compute the inverse logit. 
    res <- 1/(1 + exp(-eta))        # Recall this is our link function.
    return(res)
}

pr  <- invlogit(beta0 + beta1*x)
y   <- rbinom(n, 1, pr)
dat <- data.frame(y, x)

lm1 <- lm(y ~ x, dat)               # Fit a linear model and a binomial logistic model

glm1 <- glm(y ~ x, dat, family = binomial(link = "logit"))

tidy(lm1)

### 3.1.1 Comparing our Linear Model and Bionomial Logistic Model

 - Compare linear regression to logistic 
 - Logistic does better job of recovering betas (i.e., `1.2` and `-0.7`)

In [ ]:
tidy(lm1)

tidy(glm1)


# 4. Multivariate Binomial Logistic Regression

Suppose now we want to investigate the effect of both sex and age on survival. We use the model below. 

In [ ]:
fm2 <- glm(survived ~ sex + age, titanic_df, family = binomial(link = "logit"))

In [ ]:
tidy(fm2)

glance(fm2)

In [ ]:
predict(fm2, newdata = data.frame(sex = "female", age = 40), type = "response")

<center><h1>Poisson Regression Models in R</h1></center>

# 5. Poisson Regression

1. Form of Poisson model for single predictor

$$ \text{log}(\mu) = \beta_0 + \beta_1 x_1 $$ 

2. Link function is $\text{log}(\cdot)$

3. We use Poisson regression when we model count data   % (e.g., $y_i \in \{0, 1, 2, \dots \}$)
  - Number of offspring an individual has
  -  Number bacterial colonies in Petri dish
4. As we saw with logistic regression, we _could_ use a linear model instead (Don't do this). but our parameter estimates would be biased, and our model inaccurate

## 5.1 Poisson Distribution
<center>
<img src=images/poisson_dist.png width = "860">
</center>


## 5.2 Poisson Regression

1. As with linear and logistic regression, we can use Poisson regression to estimate effects of predictors on some outcome

2. We can also use fitted Poisson regression models to predict future values of some outcome variable given known values for the covariates

3. Frequently used for modeling rare events

### 5.2.1 Assumptions of Poisson Regression
1. Log-transformed outcomes are linearly related to predictors 
2. Observations are independent 
3. Distributional assumption:  $y_i | x_i \sim \text{Poisson}(\lambda_i)$

### 5.2.2 Assumptions of Poisson Regression (cont.)

 - Note that the assumption $y_i | x_i \sim \text{Poisson}(\lambda_i)$ has some important implications.

 - The Poisson distribution has a single parameter, $\lambda$, which is both its mean and variance. 

 - It is frequently the case we will have data where the variance greatly exceeds the mean. When this happens, it is wise to consider similar alternatives to the Poisson model    

### 5.2.3 Similar Alternatives to Poisson Models
1. Quasi-Poisson regression
2. Zero-inflated Poisson regression
3. Negative Binomial regression

### 5.2.4 Evaluation of Poisson Regression Models

 - As with logistic regression, there is no direct counterpart to the $R^2$ in linear regression

 - Poisson regression models can be compared using AIC and BIC as we saw with linear and logistic regression

### 5.2.5 Interpreting Poisson Regression Parameters
 - We can exponentiate Poisson regression parameter estimates, and then treat them multiplicative effects

# 6. Poisson Model for Number of Procedures

Suppose we want to model the number of procedure for diabetes patients admitted to the hospital. We use several Poisson models below.

In [ ]:
dia_df <- read.csv("data/diabetes_data_clean.csv")


dim(dia_df)        # dimensions of dataframe (i.e., number of rows and columns)
names(dia_df)      # get column names

## 6.1 Checking Mean and Variance

- Recall the assumptions of Poisson models

In [ ]:
mean(dia_df$num_procedures)
var(dia_df$num_procedures)

## 6.2 Fitting Poisson Regression Model

- As with binomial logistic regression, we use `glm()` function

In [ ]:
fm5 <- glm(num_procedures ~ number_diagnoses, dia_df, family = poisson(link = "log"))

In [ ]:
tidy(fm5)

glance(fm5)

### 6.2.1 Plotting our Model's Variables

In [ ]:
df_dia_subset <- sample_frac(dia_df, 0.3)

ggplot(df_dia_subset, aes(x = number_diagnoses, y = num_procedures)) +
    geom_jitter(alpha = 0.3)

## 6.3 Adding Predictor Variables

- Like linear and logistic regression, we can add arbitrary number of predictors

In [ ]:
fm6 <- glm(num_procedures ~ number_diagnoses + num_medications + time_in_hospital, 
           dia_df, 
           family = poisson(link = "log"))

In [ ]:
tidy(fm6)
glance(fm6)

### 6.3.1 Plotting our Variables

In [ ]:
ggplot(df_dia_subset, aes(x = time_in_hospital, y = num_procedures, colour = number_diagnoses)) +
    geom_jitter(alpha = 0.8) + 
    geom_smooth()

<center><h1>GLMs with Interaction Effects in R</h1></center>

# 7. Interaction Effects

- Often called "moderation", or statistical moderation effects
- One variable "behaves" differently depending on the level another
- E.g. cigarette smoking is an _increasingly strong_ predictor of lung cancer as age increases


# 8. Logistic Regression with Interaction Effect

- Using `titanic` data 
- Predicting survival using age and sex

In [ ]:
titanic_df <- read.csv("data/titanic_subset.csv")

head(titanic_df)

## 8.1 Fitting Model with Interaction Term

In [ ]:
fm3 <- glm(survived ~ sex + age + age*sex, titanic_df, family = binomial(link = "logit"))

tidy(fm3)

### 8.1.1 Plotting the Effects

In [ ]:
ggplot(titanic_df, aes(x = age, y = as.numeric(survived), color = sex)) +
    stat_smooth(method = "glm", alpha = 0.2, size = 2, aes(fill = sex)) +
    geom_point(position=position_jitter(height = 0.03, width = 0)) +
    xlab("Age") + 
    ylab("Pr(survived)")

### 8.1.2 Predictions using Fitted Model

In [ ]:
fm2 <- glm(survived ~ age + sex + age*sex, titanic_df, family = binomial(link = "logit"))

predict(fm2, newdata=data.frame(age = 50, sex = "female"), type = "response")